In [1]:
# Core Libraries:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
from io import BytesIO
from zipfile import ZipFile
import requests

# Modelling Libraries:

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Models:

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from keras.models import Model, load_model
from keras.layers import Input, Dense

In [2]:
# Loading Credit Card Fraud Dataset:

url_fraud = 'https://github.com/manugaco/ML_User_Cases/blob/master/Datasets/fraud.zip?raw=true'
zip_fraud = ZipFile(BytesIO(requests.get(url_fraud).content), 'r')
df_fraud = pd.read_csv(zip_fraud.open(zip_fraud.namelist()[0]), low_memory=False)

In [3]:
df_fraud.head()

Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  
3 -0.221929  0.062723  0.061458  123.50      0  
4  0.502292  0.219422  0.215153   69.99      0  

[5 rows x 31 columns]

In [4]:
df_fraud['Class'].value_counts().reset_index().apply(lambda x: x/x.sum())

index     Class
0    0.0  0.998273
1    1.0  0.001727

In [5]:
df_fraud = df_fraud.drop([ 'Time'], axis = 1)
df_fraud['Amount'] = StandardScaler().fit_transform(df_fraud['Amount'].values.reshape(-1, 1))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_fraud.drop('Class',1) , df_fraud['Class'], test_size=0.2, random_state=0)

In [7]:
y_test.value_counts().reset_index().apply(lambda x: x/x.sum())

index     Class
0    0.0  0.998227
1    1.0  0.001773

In [8]:
y_train.value_counts().reset_index().apply(lambda x: x/x.sum())

index     Class
0    0.0  0.998284
1    1.0  0.001716

In [9]:
X_train_nofraud = X_train[y_train==0]
X_train_fraud = X_train[y_train==1]

In [10]:
input_layer = Input(shape=(X_train.shape[1], ))
encoded = Dense(10, activation='relu')(input_layer)
decoded = Dense(X_train.shape[1], activation='relu')(encoded)
autoencoder = Model(input_layer,decoded)

In [11]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [12]:
autoencoder.fit(X_train_nofraud, X_train_nofraud, epochs = 10, batch_size=128, validation_data=(X_train_nofraud,X_train_nofraud))

Epoch 1/10
889/889 [==============================] - 5s 5ms/step - loss: 0.9191 - val_loss: 0.8422
Epoch 2/10
889/889 [==============================] - 4s 5ms/step - loss: 0.7957 - val_loss: 0.7698
Epoch 3/10
889/889 [==============================] - 4s 4ms/step - loss: 0.7639 - val_loss: 0.7595
Epoch 4/10
889/889 [==============================] - 4s 4ms/step - loss: 0.7572 - val_loss: 0.7553
Epoch 5/10
889/889 [==============================] - 4s 5ms/step - loss: 0.7533 - val_loss: 0.7504
Epoch 6/10
889/889 [==============================] - 4s 5ms/step - loss: 0.7491 - val_loss: 0.7483
Epoch 7/10
889/889 [==============================] - 6s 6ms/step - loss: 0.7480 - val_loss: 0.7474
Epoch 8/10
889/889 [==============================] - 4s 5ms/step - loss: 0.7472 - val_loss: 0.7466
Epoch 9/10
889/889 [==============================] - 4s 4ms/step - loss: 0.7465 - val_loss: 0.7472
Epoch 10/10
889/889 [==============================] - 4s 5ms/step - loss: 0.7461 - val_loss: 0.7457

In [13]:
predictions = autoencoder.predict(X_train)
mse = np.mean(np.power(X_train - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_train})
error_df.groupby('true_class').describe()

reconstruction_error                                            \
                          count       mean        std       min       25%   
true_class                                                                  
0                      227454.0   0.745711   3.232892  0.038295  0.256246   
1                         391.0  28.612698  40.956755  0.110650  4.598432   

                                              
                  50%        75%         max  
true_class                                    
0            0.405271   0.637952  512.212162  
1           10.198034  27.414136  287.808595

In [14]:
test_predictions = autoencoder.predict(X_test)
mse = np.mean((X_test - test_predictions)**2, axis=1)
y_pred = [(lambda er: 1 if er>=11.078922  else 0)(er) for er in mse]

In [15]:
acc = metrics.accuracy_score(y_test, y_pred)
print('Accuracy score =', acc)
rec = metrics.recall_score(y_test, y_pred)
print('Recall score =', rec)
prec  =metrics.precision_score(y_test, y_pred)
print('Precision score =', prec)
f1 = metrics.f1_score(y_test, y_pred)
print('F1 score =', f1)

Accuracy score = 0.9943119974719988
Recall score = 0.5247524752475248
Precision score = 0.16109422492401215
F1 score = 0.24651162790697675
